In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Hitters.csv to Hitters.csv
User uploaded file "Hitters.csv" with length 20906 bytes


In [3]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary","League", "Division", "NewLeague"], axis = 1).astype("float")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state= 42)

In [4]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [5]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
mlp_model= MLPRegressor(hidden_layer_sizes= (100,4)).fit(X_train_scaled, y_train)
mlp_model

MLPRegressor(hidden_layer_sizes=(100, 4))

In [7]:
mlp_model.n_layers_

4

In [8]:
mlp_model.hidden_layer_sizes

(100, 4)

In [9]:
mlp_model.predict(X_train_scaled)[0:5]

array([ 26.03689307, 117.24209089,  90.54083064,  19.27201453,
        16.27228882])

In [10]:
y_pred = mlp_model.predict(X_test_scaled)

In [11]:
np.sqrt(mean_squared_error(y_test, y_pred))

605.9372697138596

In [12]:
mlp_params = {'alpha' : [0.1, 0.01, 0.02, 0.005],
              'hidden_layer_sizes': [(20,20), (100,50,150), (300,200,150)],
              'activation' : ['relu', 'logistic']}

In [13]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv=10)

In [14]:
mlp_cv_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 4)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [15]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.005, 'hidden_layer_sizes': (100, 50, 150)}

In [16]:
mlp_tuned = MLPRegressor(alpha = (0.02), hidden_layer_sizes=(100,50,150))

In [17]:
mlp_tuned.fit(X_train_scaled, y_train)

MLPRegressor(alpha=0.02, hidden_layer_sizes=(100, 50, 150))

In [18]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [19]:
np.sqrt(mean_squared_error(y_test, y_pred))

354.4892790308278